In [1]:
import tensorflow as tf
import tensorflow_text
import os
import numpy as np
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

2025-03-24 14:12:58.562883: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 14:12:58.574605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742825578.587368    2087 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742825578.590933    2087 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 14:12:58.604455: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print("Please provide the following details accurately\n")

path_data = os.path.abspath(input("Relative path to the Dataset: "))

vocab_size = input("Enter max vocabulary size (press enter to leave to default 40,000): ").strip()


Please provide the following details accurately



In [3]:
if vocab_size is None or vocab_size.isnumeric() == False:
    vocab_size = 40000
else:
    vocab_size = int(vocab_size)

path_vocab=os.path.abspath(path_data+'.vocab')

In [4]:
def load_data(path):
    
    with open(path, encoding='utf-8') as file:
        lines = file.readlines()
        
        len_data=len(lines)
        context = np.empty(shape=len_data,dtype=object)
        i=0
        for line in lines:
            context[i] = line
            i+=1
    del lines
    return context

In [5]:

target_full_raw = load_data(path_data)


In [6]:
#convert to tensor
full_en = tf.data.Dataset.from_tensor_slices(target_full_raw)


I0000 00:00:1742825591.105316    2087 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [7]:
bert_tokenizer_params=dict(lower_case=False)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]


In [8]:
bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = vocab_size,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [9]:
print("Building Vocabulary, This may take some time...")
en_vocab = bert_vocab.bert_vocab_from_dataset(
    full_en.batch(1000).prefetch(2),
    **bert_vocab_args
)
print("Vocabulary build complete!")

Building Vocabulary, This may take some time...


2025-03-24 14:13:13.034753: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Vocabulary build complete!


In [10]:
en_vocab_size = len(en_vocab)

print()
print("length of vocabulary:",en_vocab_size)
print("Examples:")
print(en_vocab[:10])
print(en_vocab[100:110])
print(en_vocab[1000:1010])
print(en_vocab[-10:])



length of vocabulary: 33593
Examples:
['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '$', '%', '&']
['÷', '˚', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ']
['बाजारांत', 'वाट', 'आडमेळीं', 'हाच्या', 'तरेकवार', 'हात', 'पद्दतींत', 'पळोवपा', 'कर्मचारी', '##खातीर']
['##ढ़', '##फ़', '##य़', '##ॠ', '##।', '##ॲ', '##“', '##”', '##…', '##⅓']


In [11]:
#write vocab to file
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

write_vocab_file(path_vocab, en_vocab)